In [1]:
import pandas as pd

frame_def = pd.read_csv("/home/anca/Documents/frames/data/fixed_frame_def2.csv")
frame_names = [f.lower().replace("f:", "") for f in frame_def["frame"]]
frame_names.append("none")



just_one_frame_per_sent = ['DS3-036','DSN0-002','DSN0-007','DSN0-009','DSN0-010','DSN0-021','DSN0-024','DSN0-027',
                           'DSN0-032','DSN0-034','DSN0-036','DSN0-037','DSN0-038','DSN0-040','DSN0-046','DSN0-051',
                           'DSN0-054','DSN0-056','DSN0-058','DSN0-060','DSN0-064','DSN0-066','DSN0-070','DSN0-073',
                           'DSN0-083','DSN0-088','DSN0-091','DSN0-093','DSN0-095','DSN0-097','DSN0-100','DSN0-106',
                           'DSN0-113','DSN0-117','DSN0-126','DSN0-129','DSN0-135','DSN0-137','DSN0-138','DSN0-139',
                           'DSN0-159','DSN0-166','DSN0-168','DSN0-169','DSN0-174','DSN0-184','DSN0-188','DSN0-189',
                           'DSN0-201','DSN0-207','DSN0-208','DSN0-213','DSN0-218','DSN0-220','DSN0-221','DSN0-224',
                           'DSN0-227','DSN0-228','DSN0-230','DSN0-232','DSN0-235','DSN0-238','DSN0-242','DSN0-247',
                           'DSN0-248','DSN0-252','DSN0-256','DSN0-264','DSN0-270','DSN0-273','DSN0-279','DSN0-280',
                           'DSN0-281','DSN0-301','DSN0-303','DSN0-304','DSN0-310','DSN0-311','DSN0-313','DSN0-315',
                           'DSN0-320','DSN0-323','DSN0-330','DSN0-333','DSN0-338','DSN0-339','DSN0-341','DSN1-003',
                           'DSN1-004','DSN1-008','DSN1-009','DSN1-014','DSN1-015','DSN1-019','DSN1-021','DSN1-027',
                           'DSN1-029','DSN1-032','DSN1-034','DSN1-045','DSN1-049','DSN1-058','DSN1-059','DSN1-063',
                           'DSN1-065','DSN1-070','DSN1-083','DSN1-084','DSN1-085','DSN1-086','DSN1-094','DSN1-102',
                           'DSN1-103','DSN1-105','DSN1-107','DSN1-110','DSN1-119','DSN1-123','DSN1-126','DSN1-128',
                           'DSN1-130','DSN1-136','DSN1-139','DSN1-142','DSN1-147','DSN1-149','DSN1-151','DSN1-152',
                           'DSN1-155','DSN1-161','DSN1-163','DSN1-168','DSN1-169','DSN1-175','DSN1-178','DSN1-180',
                           'DSN1-185','DSN1-187','DSN1-189','DSN1-197','DSN1-204','DSN1-209','DSN1-220','DSN1-221',
                           'DSN1-223','DSN1-226','DSN1-227','DSN1-229','DSN1-230','DSN1-238','VID-211742','VID-199596']

In [2]:
import pdb

frames_folder = "/home/anca/Documents/frames/data/crowdsourcing/output/sentence-centric/fn_corpus/"
#frames_dataset = pd.read_csv(frames_filename)

#frames_dataset.head(n=10)
import sys
stdout = sys.stdout

sys.path.append('../../')

from defaultconfig import Configuration
class FramesConfig(Configuration):
    inputColumns = ["Input.word_phrase", "Input.frames", "Input.sentence", "Input.beg", "Input.end",
                    "Input.vid"]
    outputColumns = ["Answer.FrameType"]
    
    # processing of a closed task
    open_ended_task = False
    annotation_vector = frame_names
    
    def processJudgments(self, judgments):
        # judgments = judgments[~judgments['Input.vid'].isin(just_one_frame_per_sent)]
        # judgments = judgments.reset_index(drop=True)
        
        for col in self.outputColumns:
            judgments[col] = judgments[col].apply(lambda x: str(x).replace('|',','))
            
            judgments[col] = judgments[col].apply(lambda x: str(x).replace('None of the above.','none'))
            judgments[col] = judgments[col].apply(lambda x: str(x).replace(' ','_'))
            judgments[col] = judgments[col].apply(lambda x: str(x).lower())
        return judgments

config = FramesConfig()

# pre-process the data and create the annotation vectors
from controllers.inputController import processFile
pre_processed_results = processFile(
    root=".", directory=frames_folder, filename="",
    config=config
)


In [4]:
# %%debug

sys.stdout = stdout

# run the metrics
from models import Metrics
processed_results = Metrics.run(pre_processed_results, config)

1 iterations; max d= 0.9999 ; wqs d= 0.660383410344; sqs d= 0.569290521806; rqs d= 0.346151838391
2 iterations; max d= 0.274672673243 ; wqs d= 0.136631618727; sqs d= 0.105936167348; rqs d= 0.0301795497187
3 iterations; max d= 0.0402298033932 ; wqs d= 0.00844655300082; sqs d= 0.00478026127785; rqs d= 0.00199383661514
4 iterations; max d= 0.0199262147214 ; wqs d= 0.00281295297744; sqs d= 0.00509890500271; rqs d= 0.00156463335605
5 iterations; max d= 0.00546546807128 ; wqs d= 0.000701303463544; sqs d= 0.000281348944458; rqs d= 9.3437371847e-05
6 iterations; max d= 0.00156982155691 ; wqs d= 0.000181610761728; sqs d= 0.00034760782813; rqs d= 0.000111223247025
7 iterations; max d= 0.000520335382856 ; wqs d= 5.97969879876e-05; sqs d= 2.75424275487e-05; rqs d= 7.47517554927e-06


In [5]:
import operator

dlist = []
max_frame = []
max_fss = []
for idx in processed_results["units"].index:
    if idx != "unit_annotation_score":
        aux_dict = { k:v for k, v in processed_results["units"]["unit_annotation_score"][idx].iteritems() if v }
        dlist.append(aux_dict)
        max_frame.append(max(aux_dict.iteritems(), key=operator.itemgetter(1))[0])
        max_fss.append(max(aux_dict.values()))
#dlist.append({})
#max_frame.append("")
#max_fss.append(0)

processed_results["units"]["FSS"] = pd.Series(dlist, index=processed_results["units"].index)
processed_results["units"]["max_frame"] = pd.Series(max_frame, index=processed_results["units"].index)
processed_results["units"]["max_fss"] = pd.Series(max_fss, index=processed_results["units"].index)

#processed_results["units"].to_csv("pilot_data_out_sentence_metrics.csv")

In [6]:
processed_results["units"]

,duration,input.beg,input.end,input.frames,input.sentence,input.vid,input.word_phrase,job,metrics.avg_annotations,metrics.avg_cos_clarity,...,output.FrameType,output.FrameType.annotations,output.FrameType.cos_clarity,output.FrameType.unique_annotations,worker,uqs,unit_annotation_score,FSS,max_frame,max_fss
unit,,,,,,,,,,,,,,,,,,,,,
301KG0KX9CXSYWVXYIGKVMSJJHG2HP,789.866667,255,261,"f:Connectors,f:Cause_change,f:People,f:Hinderi...",The security team should : A : Publish a perio...,FNC-2539,safety,,17.0,0.825029,...,"{u'cause_to_experience': 0, u'taking_sides': 0...",17,0.825029,4,15,0.535843,"{u'cause_to_experience': 0.0, u'taking_sides':...","{u'cause_change': 0.00495453442325, u'being_at...",being_at_risk,0.676315
302U8RURJZDT5JFAPQV82BMQ2R0VNH,505.400000,29,40,"f:Feeling,f:Communication,f:Assessing,f:People...",Their complaint : the system discourages worki...,FNC-0830,discourages,,26.0,0.533002,...,"{u'cause_to_experience': 0, u'taking_sides': 0...",26,0.533002,11,15,0.212370,"{u'cause_to_experience': 0.0, u'taking_sides':...","{u'none': 0.0337792797442, u'experiencer_obj':...",subjective_influence,0.364921
302U8RURJZDT5JFAPQV82BMQ2RHNVQ,460.400000,49,54,"f:Leadership,f:Manipulation,f:Communication,f:...","Closer to town is Lion Rock , shaped like a li...",FNC-1652,lying,,19.0,0.857690,...,"{u'cause_to_experience': 0, u'taking_sides': 0...",19,0.857690,8,15,0.437006,"{u'cause_to_experience': 0.0, u'taking_sides':...","{u'people': 0.0417156158766, u'holding_off_on'...",posture,0.682872
304QEQWKZPWVU3E6N87WW7W5Y59O0E,200.066667,34,39,"f:Communication,f:Choosing,f:Cause_change,f:So...",The eyewitness accounts came from calls made f...,FNC-0419,calls,,21.0,0.800000,...,"{u'cause_to_experience': 0, u'taking_sides': 0...",21,0.800000,2,15,0.777464,"{u'cause_to_experience': 0.0, u'taking_sides':...","{u'communication': 0.6197835154, u'contacting'...",contacting,0.900284
304QEQWKZPWVU3E6N87WW7W5Y5A0OR,213.600000,48,54,"f:Opinion,f:Building,f:Communication,f:Categor...",The atmosphere of 19th - century Canton is bes...,FNC-1078,evoked,,26.0,0.904194,...,"{u'cause_to_experience': 0, u'taking_sides': 0...",26,0.904194,11,15,0.560957,"{u'cause_to_experience': 0.0, u'taking_sides':...","{u'building': 0.00505837991295, u'coming_up_wi...",evoking,0.823235
306W7JMRYYATTYU0E7FPJ22OA4L8B0,281.733333,60,64,"f:Leadership,f:Have_associated,f:Connectors,f:...",Such sights include the colorful `` tams '' --...,FNC-3159,worn,,22.0,0.936382,...,"{u'cause_to_experience': 0, u'taking_sides': 0...",22,0.936382,9,15,0.661335,"{u'cause_to_experience': 0.0, u'taking_sides':...","{u'wearing': 0.87332556013, u'none': 0.0316614...",wearing,0.873326
307L9TDWJY49TADR5GP2JGU7B77N3C,414.666667,126,130,"f:Aggregate,f:Communication,f:Working_on,f:Peo...","In 1844 , American explorer John C. Fremont pa...",FNC-1653,mail,,17.0,0.650945,...,"{u'cause_to_experience': 0, u'taking_sides': 0...",17,0.650945,8,15,0.196213,"{u'cause_to_experience': 0.0, u'taking_sides':...","{u'sending': 0.111839341344, u'none': 0.069681...",sent_items,0.382030
3087LXLJ6MS1OQ1DO5O69DNR64J0F5,444.666667,19,28,"f:Cause_change,f:Expensiveness,f:Travel",A popular optional excursion is an hour 's det...,FNC-1091,excursion,,15.0,0.994135,...,"{u'cause_to_experience': 0, u'taking_sides': 0...",15,0.994135,3,15,0.955060,"{u'cause_to_experience': 0.0, u'taking_sides':...","{u'cause_change': 0.00535014179609, u'travel':...",travel,0.978956
308KJXFUJRI788HWBRQOFNE56Z2TAW,668.400000,160,168,"f:Communication,f:Cause_change,f:People,f:Dama...",Foshan 's most outstanding artistic monument i...,FNC-3051,visiting,,20.0,0.948122,...,"{u'cause_to_experience': 0, u'taking_sides': 0...",20,0.948122,5,15,0.766036,"{u'cause_to_experience': 0.0, u'taking_sides':...","{u'come_together': 0.0784279845359, u'residenc...",visiting,0.921069


In [7]:
unique_sentences = processed_results["units"]["input.sentence"]
unique_frames = config.annotation_vector

sentences = []
frames = []
words = []
sent_text = []
fss = []

for idx in processed_results["units"].index:
    sent_frames = processed_results["units"]["input.frames"][idx].split(",")
    sent_frames = [x.replace('f:','').lower() for x in sent_frames]
    for frame in sent_frames:
        sentences.append(processed_results["units"]["input.vid"][idx])
        words.append(processed_results["units"]["input.word_phrase"][idx])
        sent_text.append(processed_results["units"]["input.sentence"][idx])
        frames.append(frame)
        if frame in processed_results["units"]["FSS"][idx].keys(): 
            fss.append(processed_results["units"]["FSS"][idx][frame])
        else:
            fss.append(0.0)
    #frame = "none"
    #sentences.append(processed_results["units"]["input.vid"][idx])
    #words.append(processed_results["units"]["input.word_phrase"][idx])
    #sent_text.append(processed_results["units"]["input.sentence"][idx])
    #frames.append(frame)
    #if frame in processed_results["units"]["FSS"][idx].keys(): 
    #    fss.append(processed_results["units"]["FSS"][idx][frame])
    #else:
    #    fss.append(0.0)

In [8]:
frame_sentence_score = pd.DataFrame({
    "sent_id" : sentences,
    "sentence" : sent_text,
    "word" : words,
    "fss_sent_centric" : fss,
    "frame" : frames
})

#frame_sentence_score.set_index([frame_sentence_score["sent_id"], frame_sentence_score["frame"]])
frame_sentence_score

,frame,fss_sent_centric,sent_id,sentence,word
0,connectors,0.004955,FNC-2539,The security team should : A : Publish a perio...,safety
1,cause_change,0.004955,FNC-2539,The security team should : A : Publish a perio...,safety
2,people,0.000000,FNC-2539,The security team should : A : Publish a perio...,safety
3,hindering,0.000000,FNC-2539,The security team should : A : Publish a perio...,safety
4,supply,0.000000,FNC-2539,The security team should : A : Publish a perio...,safety
5,commerce_scenario,0.000000,FNC-2539,The security team should : A : Publish a perio...,safety
6,commitment,0.000000,FNC-2539,The security team should : A : Publish a perio...,safety
7,connecting_architecture,0.000000,FNC-2539,The security team should : A : Publish a perio...,safety
8,quantity,0.000000,FNC-2539,The security team should : A : Publish a perio...,safety
9,being_at_risk,0.676315,FNC-2539,The security team should : A : Publish a perio...,safety


In [9]:
frame_sentence_score.to_csv(
    "/home/anca/Documents/frames/data/crowdsourcing/output/sentence-centric/fn_corpus.csv",
    index = False
)

In [10]:
processed_results["annotations"].to_csv(
    "/home/anca/Documents/frames/data/crowdsourcing/output/sentence-centric/fn_corpus_fqs.csv"
)

In [11]:
processed_results["units"].to_csv(
    "/home/anca/Documents/frames/data/crowdsourcing/output/sentence-centric/fn_corpus_sqs.csv"
)

In [12]:
processed_results["workers"].to_csv(
    "/home/anca/Documents/frames/data/crowdsourcing/output/sentence-centric/fn_corpus_wqs.csv"
)

In [13]:
processed_results["judgments"].to_csv(
    "/home/anca/Documents/frames/data/crowdsourcing/output/sentence-centric/fn_corpus_judgments.csv"
)

In [34]:
import pdb
from itertools import combinations
# print pre_processed_results["judgments"][1:10]


unit_judg_len = pre_processed_results["judgments"].groupby("unit").size()
comb_dict = dict()
for idx in unit_judg_len.index:
    # print unit_judg_len[idx]
    comb_dict[idx] = combinations(range(unit_judg_len[idx]), 14)
    print pre_processed_results["judgments"].loc[pre_processed_results["judgments"]["unit"] == idx].groupby("output.FrameType").size()

TypeError: unhashable type: 'OrderedCounter'